In [1]:
import sys
import os
import random
import time
from googleapiclient.discovery import build

sys.path.append(os.path.abspath(".."))

from src.youtube_summariser import YouTubeHelper, cli, llm_client

In [11]:
# --- Initialization ---
youtube_key = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=youtube_key)

channels = {
    "BWB - Business With Brian": "UULFyqlbzLoYtpqDXwRI9Yh5LA",
    "Chris Sain": "UULFrTFPf6rq5OUSWb7ILW9trg",
    "Asymmetric Investing by Travis Hoium": "UULFM2udYo4m-_uQfbfLGwf6mA",
    "Felix & Friends (Goat Academy)": "UULFJtfma0mE_XrBAD9uakcjfA"
}

start_date = "2026-01-25"
end_date = "2026-01-27"

individual_summary_prompt = """
You are a specialized Equity Data Extractor. Your sole task is to identify every stock ticker mentioned in the provided transcript where a specific reason or thesis is given.

Instructions:

    Scan the text for company names or tickers (e.g., 'Bloom Energy' or 'BE').

    For each, extract:

        Ticker: [TICKER]

        Sentiment: [Bullish/Bearish/Neutral]

        Thesis: [The specific technical or fundamental reason given]

        Metrics: [Any specific numbers mentioned: prices, P/E, percentages]

Constraint: If a stock is mentioned without a reason (e.g., 'I also like Apple'), ignore it. Do not paraphrase heavily; stay true to the speaker's specific logic.
"""

# group_summary_prompt = """
# You are a Senior Equity Research Analyst. I will provide you with several individual stock extraction reports from various YouTube channels.

#     Your Task:

#         De-duplicate: If multiple sources mentioned the same stock, merge their insights into a single entry.

#         Attribution: List all channels that mentioned that specific stock in the 'Source(s)' field.

#         Formatting: Use the following structure for every unique ticker:

#         [TICKER] ([Company Name])

#             Sentiment: [Merged Sentiment]

#             The Thesis: [Consolidated 2-3 sentence summary. If sources disagree, note both points of view.]

#             Source(s): [e.g., Chris Sain, BWB]

#             Key Metrics & Data: [List all numbers provided across all sources. If none, state 'None provided'.]

#     Constraint: Maintain an objective, institutional tone.
#     """


group_summary_prompt = """
You are a Senior Analyst. Your task is to take a list of individual stock mentions and organize them into a final 'State of the Market' report.

    CRITICAL RULE: You must include EVERY unique ticker found in the provided list. Do not omit any stock, even if the thesis seems minor.

    Instructions:

        If the same ticker appears in multiple reports, merge them into one entry and list all unique sources in the 'Source(s)' field.

        Use this exact structure for every stock:

        [TICKER] ([Company Name])

            Sentiment: [Merged Sentiment]

            The Thesis: [2-3 sentence summary. Combine the points from all sources.]

            Source(s): [List all source channels]

            Key Metrics: [List all numbers found across all reports]

    Constraint: Do not 'summarize' the list into a shorter version. This is a comprehensive extraction report. Every data point must be preserved.
"""

In [7]:
# --- Phase 1: High Efficiency Collection ---

def collect_videos_from_channels(channels, start_date, end_date):
    collected_videos = []
    for name, cid in channels.items():
        
        print(f"Scanning {name} via Playlist...")
        try:
            request = youtube.playlistItems().list(
                part="snippet",
                playlistId=cid,
                maxResults=10  # 10 results for only 1 unit!
            )
            response = request.execute()
            
            # items is a list of dictionaries
            items = response.get('items', [])
    
            for item in items:
                # Check for standard dict access
                snippet = item['snippet']
                v_title = snippet['title']
                published_at = snippet['publishedAt']                
                v_id = snippet['resourceId']['videoId']
                
                # Date Filter (Match your specific range)
                if start_date <= published_at <= end_date:
                    collected_videos.append({
                        "v_id": v_id,
                        "title": v_title,
                        "channel": name
                    })
                    print(f"  ✅ Found: {v_title}")
    
        except Exception as e:
            print(f"❌ Failed {name}: {e}")

    return collected_videos



def get_transcripts_for_videos(collected_videos):    
    # --- 2: Pull Transcripts for Successes ---
    master_data = []
    
    print(f"\nFound {len(collected_videos)} videos. Fetching transcripts...")
    
    for video in collected_videos:
        try:
            raw = YouTubeHelper.get_transcript(video['v_id'])
            if isinstance(raw, str):
                # If your helper already joins the text into a string, use it directly
                full_text = raw 
            else:
                # If it's the raw list of dicts, join it here
                full_text = "\n".join([t['text'] for t in raw])
                
            master_data.append({
                "channel": video['channel'],
                "title": video['title'],
                "transcript": full_text
            })
            print(f"✅ Success: {video['title']}")
            wait_time = random.uniform(5, 12)
            print(f"Sleeping for {wait_time:.2f}s to avoid IP block...")
            time.sleep(wait_time)
        except Exception as e:
            print(f"❌ Failed transcript for {video['title']}")
            print(e)
            raise
    return master_data


def get_individual_summaries(master_data):
    individual_summaries = []
    
    # --- PHASE 1: INDIVIDUAL SCANNING ---
    for entry in master_data:
        print(f"Scanning {entry['title']}...")
        
        # We pass the 'channel' name into the prompt context so Gemini knows the source
        user_msg = f"VIDEO SOURCE: {entry['channel']}\n\nTRANSCRIPT:\n{entry['transcript']}"

        llm = llm_client.LLMClient(provider="google")
        individual_summary = llm.chat(system_prompt=individual_summary_prompt, user_message=user_msg) 
        
        # Save this summary along with the source channel for the next step
        individual_summaries.append(f"SOURCE: {entry['channel']}\n{individual_summary}")
        
        # Respect Free Tier RPM limits (approx 4-6 seconds between requests)
        time.sleep(10)
    return individual_summaries


def get_master_summary(individual_summaries):
    combined_input_for_master = "\n\n---\n\n".join(individual_summaries)
    llm = llm_client.LLMClient(provider="google")
    master_summary = llm.chat(system_prompt=group_summary_prompt, user_message=combined_input_for_master) 
    return master_summary

    

In [4]:
collected_videos = collect_videos_from_channels(channels, start_date, end_date)
collected_videos

Scanning BWB - Business With Brian via Playlist...
  ✅ Found: I Was Ranked #1: My Top Stock Picks for 2026 📈
Scanning Chris Sain via Playlist...
  ✅ Found: The Stock Market Is About To Go INSANE TOMORROW
Scanning Asymmetric Investing by Travis Hoium via Playlist...
Scanning Felix & Friends (Goat Academy) via Playlist...
  ✅ Found: Leaked: Trump’s Metals Law - Most Investors Aren't Ready!


[{'v_id': 'OsEeetpStF4',
  'title': 'I Was Ranked #1: My Top Stock Picks for 2026 📈',
  'channel': 'BWB - Business With Brian'},
 {'v_id': 'xbeuzjvk78Q',
  'title': 'The Stock Market Is About To Go INSANE TOMORROW',
  'channel': 'Chris Sain'},
 {'v_id': 'b_fWN5XNfUw',
  'title': "Leaked: Trump’s Metals Law - Most Investors Aren't Ready!",
  'channel': 'Felix & Friends (Goat Academy)'}]

In [5]:
master_data = get_transcripts_for_videos(collected_videos)
master_data


Found 3 videos. Fetching transcripts...
✅ Success: I Was Ranked #1: My Top Stock Picks for 2026 📈
Sleeping for 9.20s to avoid IP block...
✅ Success: The Stock Market Is About To Go INSANE TOMORROW
Sleeping for 8.71s to avoid IP block...
✅ Success: Leaked: Trump’s Metals Law - Most Investors Aren't Ready!
Sleeping for 11.49s to avoid IP block...


[{'channel': 'BWB - Business With Brian',
  'title': 'I Was Ranked #1: My Top Stock Picks for 2026 📈',
  'transcript': "It's the start of a new year and like\nmany other YouTubers, it's my one chance\nwhere I get to pick my top stocks going\ninto 2026. But this year, I've got to\nadmit, I feel a little more added\npressure because of this video right\nover here from Adriconomics where he\ncompared several different YouTubers\npicks for all of 2025. It happens to be\na great video if you haven't already\nseen it. And to make a long story short,\nat the end of the year, I wound up\ntaking the top spot. First off, I\nabsolutely love his video concept. And\nsecond, I'm pretty impressed that most\nof these YouTubers wound up beating the\nS&P 500. Hi everyone. If you're new to\nthe channel, I'm Brian, and I happen to\nretire six years ago at the age of 46,\nwhich is only after a 15-year career at\ncompanies like Target and Amazon. And if\nyou happen to do the math, then yes, I\nstarted my ca

In [10]:
# individual_summaries = get_individual_summaries(master_data)
for summary in individual_summaries:
    print(summary)

SOURCE: BWB - Business With Brian
**Ticker:** ANET (Arista Networks)
**Sentiment:** Bullish
**Thesis:** Arista is a critical provider of high-speed networking fabric for AI data centers. As AI clusters scale, networking becomes the primary bottleneck rather than compute. The company is benefiting from hyperscaler refresh cycles and exhibits significant operating leverage where earnings are compounding faster than revenue.
**Metrics:** Revenue expanded from $2.3B (2020) to estimated $8.5B (2025) (~30% CAGR); net margins expanded from 27% to 42%+; Q3 2025 revenue of $2.31B; net income up 37% YoY; free cash flow margins exceed 50%; $9.5B in cash/investments vs. $60M in debt; 36% analyst upside.

**Ticker:** APLD (Applied Digital)
**Sentiment:** Bullish
**Thesis:** The company focuses on AI data center compute density and low power costs by securing long-duration electricity in power-rich regions (North Dakota). This provides pricing certainty for hyperscalers via 10-15 year leases. The gr

In [14]:
import importlib
import src.youtube_summariser as youtube_summariser

importlib.reload(youtube_summariser)

from src.youtube_summariser import YouTubeHelper, cli, llm_client

In [15]:
master_summary = get_master_summary(individual_summaries)
print(master_summary)

**State of the Market Report**

**AMD (Advanced Micro Devices)**
*   **Sentiment:** Bullish
*   **The Thesis:** The market is currently undervaluing AMD's growth potential, particularly evidenced by a low PEG ratio. Revenue is shifts toward higher-margin data center products, and while non-cash amortization from the Xilinx acquisition masks true earnings power, free cash flow is improving sharply.
*   **Source(s):** BWB - Business With Brian
*   **Key Metrics:** Revenue grew from $10B (2020) to $34B (2025); PEG ratio of 0.47; data center revenue grew 60%+ YoY in 2025; free cash flow hit $2.4B; $3B annual non-cash amortization; 27% analyst upside.

**AMZN (Amazon)**
*   **Sentiment:** Bullish
*   **The Thesis:** AWS growth is re-accelerating toward 20%, a historical signal for a stock price breakout. The business exhibits massive operating leverage, reaching record profits and free cash flow which allows them to fund data center expansions internally.
*   **Source(s):** BWB - Business W

In [16]:
len(master_summary)

9710

In [17]:
with open("flash_report.txt", "w", encoding="utf-8") as f:
    f.write(master_summary)